<a href="https://colab.research.google.com/github/WillieHabim/COMP551-A2/blob/main/COMP551_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1.1

##Task 1.1

Get Data

In [ ]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 

In [ ]:
!tar --gunzip --extract --file=aclImdb_v1.tar.gz 

In [ ]:
import pandas as pd

words_table = pd.read_csv('aclImdb/imdb.vocab', header=None)
words_table.head()

,0
0,the
1,and
2,a
3,of
4,to


In [ ]:
!pip install -U libsvm-official

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
words = words_table.to_numpy()
words.shape

(89527, 1)

In [ ]:
valid_indices = [1 for i in range(words.shape[0])]
len(valid_indices)

89527

Remove rare and stopwords

In [ ]:
import scipy
from libsvm.svmutil import *
y, x = svm_read_problem('aclImdb/train/labeledBow.feat', return_scipy = True) # y: ndarray, x: csr_matrix
appearances = (x != 0).sum(0)
ratios = appearances/y.shape[0]
valid_indices = np.where((ratios < .01) | (ratios > 0.5), False, True).flatten()
invalid_indices = np.logical_not(valid_indices)

In [ ]:
valid_indices = np.array(valid_indices, dtype=bool)
valid_words = words[valid_indices, :]
X_imdb = x[:, valid_indices]
y_imdb = y

Find 500 top features/words by absolute z-score 

In [ ]:
from sklearn.preprocessing import StandardScaler
import math

scaler = StandardScaler()
x_standard = scaler.fit_transform(X_imdb.toarray())
y_standard = scaler.fit_transform(y.reshape(-1,1)).flatten()

z = (np.dot(x_standard.T, y_standard))/math.sqrt(y.shape[0])
best_features_imdb = np.argsort(abs(z))[::-1][:100]

##Task 1.2

Getting best features from news dataset

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mutual_info_score
from heapq import heappush, heappushpop

bunch = fetch_20newsgroups(subset="train", remove=(['headers', 'footers', 'quotes']))
count_vect = CountVectorizer()
X_train_total_counts = count_vect.fit_transform(bunch.data)

# 4 categories:
# -------
# 1) rec.motorcycles 8
# 2) soc.religion.christian 15
# 3) sci.med 13
# 4) comp.windows.x 5

category_indices = np.where((bunch.target == 8) | (bunch.target == 5) |(bunch.target == 15) |(bunch.target == 13), True, False).tolist()
X_train_counts = X_train_total_counts[category_indices, :]

# One-hot Encoding
# -----------------
dic = {8:0,15:1,13:2,5:3}
y_news = np.zeros((bunch.target[category_indices].size, 4))
y_class = lambda x: dic[x]
y_news[np.arange(bunch.target[category_indices].size), list(map(y_class, bunch.target[category_indices]))] = 1

ratios = ((X_train_counts != 0).sum(0))/y_news.shape[0]
valid_indices = np.where((ratios < .01) | (ratios > 0.5), False, True).flatten()
valid_indices = np.array(valid_indices, dtype=bool)

X_news = X_train_counts[:, valid_indices]
best_features_news = set()

for i in range(4):
  minheap = []
  for j in range(X_news.shape[1]):
    mis = mutual_info_score(y_news[:,i], X_news[:,j].toarray().flatten())
    if len(minheap) < 100:
      heappush(minheap, (mis,j))
    elif minheap[0][0] < mis:
      heappushpop(minheap, (mis,j))
  
  new_words = set([i for (_,i) in minheap])
  best_features_news.update(new_words)

#Task 2

Logistic Regression

In [ ]:
logistic = lambda a: 1 / (1 + np.exp(-a))
from scipy.sparse import hstack, csr_matrix

class LogisticRegression:
    
    def __init__(self, add_bias=True, learning_rate=.1, epsilon=1e-4, max_iters=1e5, verbose=False):
        self.add_bias = add_bias
        self.learning_rate = learning_rate
        self.epsilon = epsilon                        
        self.max_iters = max_iters                    
        self.verbose = verbose

    def gradient(self, x, y):
        yh = logistic(np.dot(x, self.w))    
        grad = np.dot(x.T, yh - y)/x.shape[0]        
        return grad
        
    def fit(self, x, y):
        print(x.shape)
        if x.ndim == 1:
            x = x[:, None]
        print(x.shape)
        if self.add_bias:
            x = np.column_stack([x,np.ones(x.shape[0])])
        print(x.shape)
        self.w = np.zeros(x.shape[1])
        
        g = np.inf 
        t = 0
        
        while np.linalg.norm(g) > self.epsilon and t < self.max_iters:
            g = self.gradient(x, y)
            self.w = self.w - self.learning_rate * g 
            t += 1
        
        if self.verbose:
            print(f'terminated after {t} iterations, with norm of the gradient equal to {np.linalg.norm(g)}')
            print(f'the weight found: {self.w}')
        return self
    
    def predict(self, x):
        if x.ndim == 1:
            x = x[:, None]
        if self.add_bias:
            x = np.column_stack([x,np.ones(x.shape[0])])
        y_hat = logistic(np.dot(x,self.w))            
        return y_hat

Training on IMDB Data

In [ ]:
model = LogisticRegression()
X_imdb_bf = X_imdb[:,best_features_imdb]
model.fit(X_imdb_bf.toarray(), y_imdb)

(25000, 100)
(25000, 100)
(25000, 101)
